# Урок 6. Расчёт длительности А/B теста + продвинутые топики

In [47]:
# импорт библиотек
from statsmodels.stats.power import tt_ind_solve_power, zt_ind_solve_power
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats.meta_analysis import effectsize_smd
from typing import Union
import plotly.graph_objects as go
from scipy import stats
from math import asin
import numpy as np

## Задание 1.
_Продакт на главной mail.ru решил протестировать в рекомендательной ленте контента вместо карточек со статьями видеоплеер с короткими видео._  
Нынешний таймспент на юзера в день в среднем равен **25** минут, а стандартная ошибка (SD) равна **156**.  
Мы предполагаем, что в новой версии таймспент на юзера в день изменится на **10%**.  
Средний трафик **20000** человек в день.  
Посчитайте сколько дней необходимо держать эксперимент при **$\alpha$ = 5%** и **$\beta$ = 20%**.

In [48]:
#Функция расчёта размера эффекта (effect_size) для непрерывных метрик
def calc_continuous_effect_size(mean_1: Union[float, int],
                                std_1: Union[float, int],
                                mean_2: Union[float, int],
                                std_2: Union[float, int],
                                nobs_1: int = 10_000,
                                nobs_2: int = 10_000):
    
    es_formula = (mean_1 - mean_2) / ((std_1**2 + std_2**2) / 2) ** 0.5
    es_import = effectsize_smd(mean_1, std_1, nobs_1, mean_2, std_2, nobs_2)[0]
    
    return es_formula, es_import 

In [49]:
#Функция расчёта минимально необходимой выборки * 2(для теста и контроля) для непрерывной метрики
def calc_sample_size_continuous(effect_size: float,
                                alpha: float = .05,
                                beta: float = .2,
                                ratio: Union[float, int] = 1):
    
    n = tt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio,
                  )
    return int(n * 2)

По условию таймспент 25 мин, если таймспент на юзера в день изменится на 10%, =>  новый таймспент 27,5 мин.  
Стандартная ошибка (SD) равна 156.

In [50]:
# Расчитываем effect_size.

mu_control, mu_test = 25, 27.5
std_control, std_test = 156, 156

effect_size = calc_continuous_effect_size(mean_1=mu_control, std_1=std_control, mean_2=mu_test, std_2=std_test)[1]
print(f"Размер эффекта (effect_size) для непрерывных метрик составляет {effect_size:.6f}.")

Размер эффекта (effect_size) для непрерывных метрик составляет -0.016025.


In [51]:
# Расчитываем минимально необходимую выборку для оценки изменений.

selection = calc_sample_size_continuous(effect_size=effect_size, alpha=.05, beta=.2)
print(f"Общий размер выборки {selection} человек.")

Общий размер выборки 122257 человек.


По условию cредний трафик 20000 человек в день.

In [52]:
# Расчитаем количество дней для проведения теста.
days = selection/20000

print(f"Необходимо провести эксперимент в течение {days} дней")

Необходимо провести эксперимент в течение 6.11285 дней


Полученные 6,11285 дней округляем в большую сторону => ~ 7 дней.

**Ответ:**  
 
При $\alpha$ = 5% и $\beta$ = 20% необходимо держать эксперимент **~ 7** дней.

## Задание 2.

_Наша продуктовая команда в ecommerce магазине планирует запустить тест, направленный на ускорение загрузки сайта._   
Одна из основных метрик bounce rate в GA = **40%**.  
Мы предполагаем, что при оптимизации сайта она изменится минимум на **20%** от текущего показателя.  
Средний трафик **4000** человек в день.  
Посчитайте сколько нам нужно дней держать эксперимент при $\alpha$ = **5%** и $\beta$ = **20%**.

In [53]:
#Функция расчёта размера эффекта (effect_size) для пропорций
def calc_propotion_effect_size(conv_1: int, nobs_1: int, conv_2: int, nobs_2: int):
    prob_1, prob_2 = conv_1 / nobs_1, conv_2 / nobs_2
    
    es_formula = 2  * asin(np.sqrt(prob_1)) - 2 * asin(np.sqrt(prob_2))
    es_import = proportion_effectsize(prob_1, prob_2)
    
    return es_formula, es_import

In [54]:
#Функция расчёта минимально необходимой выборки * 2(для теста и контроля) для пропорций
def calc_sample_size_proportion(effect_size: float,
                                alpha: float = .05,
                                beta: float = .2,
                                ratio: Union[float, int] = 1):
    
    n = zt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio,
                  )
    return int(n * 2)

Метрика bounce rate в GA = 40%.  
При оптимизации сайта она изменится минимум на 20% от текущего показателя. =>  
40% х 0,2 = 8%  
40% - 8% = 32% =>  
H0=40%   # bounce rate в GA  
H1=32%   # bounce rate в GA  

 
Найдем размер выборки для каждой гипотезы, при помощи калькулятора  
https://www.evanmiller.org/ab-testing/sample-size.html


![Sem6jpg1.jpg](Sem6jpg1.jpg)


![Sem6jpg2.jpg](Sem6jpg2.jpg)

Размер минимально необходимой выборки **592** человека на каждую гипотезу.

In [55]:
# Расчитываем размер эффекта (effect_size).
conv_1, conv_2 = 40, 32
nobs_1, nobs_2 = 592, 592
prob_1, prob_2 = conv_1 / nobs_1, conv_2 / nobs_2
print(f'{prob_1:.4f}, {prob_2:.4f}')
effect_size = calc_propotion_effect_size(conv_1=conv_1, nobs_1=nobs_1, conv_2=conv_2, nobs_2=nobs_2)[0]
print(f"Размер эффекта (effect_size) для пропорций составляет {effect_size:.6f}.")

0.0676, 0.0541
Размер эффекта (effect_size) для пропорций составляет 0.056630.


In [56]:
# Расчитываем минимально необходимую выборку
selection = calc_sample_size_proportion(effect_size=effect_size, alpha=.05, beta=.2)

print(f"Общий размер выборки {selection} человек.")

Общий размер выборки 9789 человек.


По условию средний трафик 4000 человек в день.

In [57]:
# Расчитаем количество дней.
days = selection/4000

print(f"Необходимо провести эксперимент в течение {days} дней")

Необходимо провести эксперимент в течение 2.44725 дней


Полученные 2,44725 дней округляем в большую сторону => ~ 3 дня.

**Ответ:**  
 
При $\alpha$ = 5% и $\beta$ = 20% необходимо держать эксперимент **~ 3** дня.